In [1]:
import mujoco
import mujoco_viewer

# scene.xml includes the robot model and a simple environment
model = mujoco.MjModel.from_xml_path("../src/assets/universal_robots_ur5e/scene.xml")

# Contains the state of the model (time: .time, pos: .qpos, vel: .qvel) 
data = mujoco.MjData(model)

In [2]:
# Print out the name of the body accessors
try:
    model.geom()
except KeyError as e:
    print(f"model.geom: {e}")
    
try:
    data.geom()
except KeyError as e:
    print(f"data.geom: {e}")
    
try:
    data.body()
except KeyError as e:
    print(f"data.body: {e}")
    
    
trunk_id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_BODY.value, "forearm_link")
# print(data.body(trunk_id))

print(f"actuator control range: {model.actuator_ctrlrange}")

model.geom: "Invalid name ''. Valid names: ['floor']"
data.geom: "Invalid name ''. Valid names: ['floor']"
data.body: "Invalid name ''. Valid names: ['base', 'forearm_link', 'shoulder_link', 'target', 'upper_arm_link', 'world', 'wrist_1_link', 'wrist_2_link', 'wrist_3_link']"
actuator control range: [[-6.2831  6.2831]
 [-6.2831  6.2831]
 [-3.1415  3.1415]
 [-6.2831  6.2831]
 [-6.2831  6.2831]
 [-6.2831  6.2831]]


In [3]:
print(f"Simulated time: {data.time} sec")
print(f"Default timestep: {model.opt.timestep} sec")
print(f"Degrees of freedom: {model.nv=}")

print(f"{len(data.qpos)=} {data.qpos=}")
print(f"{len(data.qvel)=} {data.qvel=}")
print(f"{len(data.xpos)=} {data.xpos=}")
print(f"{len(data.site_xpos)=} {data.site_xpos=}")

Simulated time: 0.0 sec
Default timestep: 0.002 sec
Degrees of freedom: model.nv=6
len(data.qpos)=6 data.qpos=array([0., 0., 0., 0., 0., 0.])
len(data.qvel)=6 data.qvel=array([0., 0., 0., 0., 0., 0.])
len(data.xpos)=9 data.xpos=array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])
len(data.site_xpos)=2 data.site_xpos=array([[0., 0., 0.],
       [0., 0., 0.]])


In [4]:
import mujoco_viewer.mujoco_viewer
import numpy as np
import time
# Setup viewer
viewer = mujoco_viewer.MujocoViewer(model, data)
# Reset the simulation
mujoco.mj_resetDataKeyframe(model, data, 0)


print([mujoco.mj_id2name(model, mujoco.mjtObj.mjOBJ_ACTUATOR.value, i)
    for i in range(len(data.qfrc_actuator))])

dof_position_limit_multiplier = 0.9 # The % of the range that is not penalized
ctrl_range_offset = 0.5 * (1 - dof_position_limit_multiplier) * (model.actuator_ctrlrange[:, 1] - model.actuator_ctrlrange[:, 0])
_soft_ctrl_range = np.copy(model.actuator_ctrlrange)
_soft_ctrl_range[:, 0] += ctrl_range_offset
_soft_ctrl_range[:, 1] -= ctrl_range_offset

print(f"{model.jnt_range=}")

for _ in range(1000):
    if viewer.is_alive:
        mujoco.mj_step(model, data)

        print(f"{data.qfrc_actuator[-12:]=}")
        
        print("\n")
        print(data.act)
        data.qpos = model.key_qpos
        print(data.xpos[-1])
        
        viewer.render()
        time.sleep(0.01)        

    else:
        break

viewer.close()

['shoulder_pan', 'shoulder_lift', 'elbow', 'wrist_1', 'wrist_2', 'wrist_3']
model.jnt_range=array([[-6.28319,  6.28319],
       [-6.28319,  6.28319],
       [-3.1415 ,  3.1415 ],
       [-6.28319,  6.28319],
       [-6.28319,  6.28319],
       [-6.28319,  6.28319]])
data.qfrc_actuator[-12:]=array([0., 0., 0., 0., 0., 0.])


[]
[0.5 0.  0.5]
data.qfrc_actuator[-12:]=array([-4.85156742e-01, -1.25748419e+00, -1.59207557e+01,  3.88477413e-01,
        8.41130884e-03, -1.60071702e-04])


[]
[0.5 0.  0.5]
data.qfrc_actuator[-12:]=array([-3.39397117e+00, -9.36687523e+00, -7.65234982e+00, -2.66799387e+00,
        4.25741625e-02, -7.99981485e-04])


[]
[0.5 0.  0.5]
data.qfrc_actuator[-12:]=array([-1.99665560e-01, -8.31637352e+00, -1.87726739e+01,  1.05053594e+00,
       -9.51413012e-02,  1.85166674e-03])


[]
[0.5 0.  0.5]
data.qfrc_actuator[-12:]=array([-2.29264800e+00, -1.38712431e+01, -8.98392879e+00, -3.71552055e+00,
        1.25137462e-01, -2.53725644e-03])


[]
[0.5 0.  0.5]
data.qfrc_act

In [5]:
import numpy as np

a = np.array([3, 4])

print(np.linalg.norm(a))

5.0


: 